In [1]:
import warnings
warnings.filterwarnings("ignore")
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.mol_structure import draw_one_mol, draw_graphs
from evaluation.new_structural_metric import symmetric_graph_set_distance,atom,cycle,neighborhood
from evaluate import evaluate
#import utils.graph_generators as gen
import torch
import networkx as nx
import os 
import sys
import json
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
import rdkit
import numpy as np
from evaluation.utils import get_data,get_graph_data,  get_mock_data,remove_empty_graphs_and_targets,\
    get_generated_data,preprocess,get_generated_graph_data,_preprocess



# Rewired

In [2]:
import pandas as pd
def get_rewired_graph_data(name,percent):
   try:
     generated_graphs=pd.read_pickle(f'data/rewired/train1/{name}/{name}_rewired_with_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/rewired/train1/{name}/{name}_perturbed_with_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
        for n,i in enumerate(g.nodes(data=True)):
          try:
            g.nodes[n]['label']=int(g.nodes[n]['label'])
          except: None
          try:
           g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
          except: None
          try:
             g.nodes[n]['attr']=int(g.nodes[n]['true_label'])
          except: None
          del g.nodes[n]['target']

   return  generated_graphs, np.array(targets)

def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict

In [4]:
train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data('synthetic_1_50', path=r'data/graphs/datasets')
a=get_rewired_graph_data('synthetic_1_50',0)
a[0][0].nodes(data=True)

NodeDataView({0: {'label': 1, 'true_label': 1, 'attr': 1}, 1: {'label': 0, 'true_label': 0, 'attr': 0}, 2: {'label': 2, 'true_label': 2, 'attr': 2}, 3: {'label': 0, 'true_label': 0, 'attr': 0}, 4: {'label': 1, 'true_label': 1, 'attr': 1}, 5: {'label': 1, 'true_label': 1, 'attr': 1}, 6: {'label': 0, 'true_label': 0, 'attr': 0}, 7: {'label': 2, 'true_label': 2, 'attr': 2}, 8: {'label': 2, 'true_label': 2, 'attr': 2}, 9: {'label': 1, 'true_label': 1, 'attr': 1}, 10: {'label': 2, 'true_label': 2, 'attr': 2}, 11: {'label': 0, 'true_label': 0, 'attr': 0}, 12: {'label': 2, 'true_label': 2, 'attr': 2}, 13: {'label': 2, 'true_label': 2, 'attr': 2}, 14: {'label': 1, 'true_label': 1, 'attr': 1}, 15: {'label': 2, 'true_label': 2, 'attr': 2}, 16: {'label': 0, 'true_label': 0, 'attr': 0}, 17: {'label': 0, 'true_label': 0, 'attr': 0}, 18: {'label': 0, 'true_label': 0, 'attr': 0}, 19: {'label': 0, 'true_label': 0, 'attr': 0}, 20: {'label': 1, 'true_label': 1, 'attr': 1}, 21: {'label': 1, 'true_label':

In [3]:


structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=['0','5','10','25','33','40','50','66','75','90','100']
metrics={}
for generator in ['rewired']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                a=[]
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_rewired_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))


# Pertubed with ER

In [2]:

import pandas as pd
def get_corrupted_graph_data(name,percent):
   try:
     generated_graphs=pd.read_pickle(f'data/corrupted/train1/{name}/{name}_corrupted_with_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/corrupted/train1/{name}/{name}_perturbed_with_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
        for n,i in enumerate(g.nodes(data=True)):
          try:
            g.nodes[n]['label']=int(g.nodes[n]['label'])
          except: None
          try:
           g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
          except: None
          try:
             g.nodes[n]['attr']=[int(l) for l in list(g.nodes[n]['attr'])]
          except: None
          del g.nodes[n]['target']
   return  generated_graphs, np.array(targets)


def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict

In [5]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural']
#leave it blank to compute all the structural structures 
generators=['hiervae','wgan','stgg','moflow','gdss','jtnn','swingnn']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    a=[]
                    print(dataset_name+split, generator)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_corrupted_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

ames_50 corrupted
Now computing classifier based metrics
Cannot compute the classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))


In [12]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['corrupted']:
    for dataset_name in ['synthetic_2','synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                    generated_graphs, generated_targets= get_corrupted_graph_data(dataset_name+split,int(percent*100))
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)
                    metrics=dict_mean(a)
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)


synthetic_2_50 0
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.82495 0.82805 0.8065000000000001 0.82805
{'activations_time': 0.03967002232869466, 'fid': 0.4230917669382945, 'fid_time': 0.043381118774414064, 'kid': 0.006520223617553711, 'kid_time': 0.3918835004170736, 'precision': 0.9776666666666666, 'recall': 0.9808333333333333, 'f1_pr': 0.979224913839528, 'precision_time': 0.15288906097412108, 'recall_time': 0.15288906097412108, 'f1_pr_time': 0.15288906097412108, 'density': 0.9774999999999998, 'coverage': 0.9488333333333332, 'f1_dc': 0.9617888190885137, 'density_time': 0.10285983085632325, 'coverage_time': 0.10285983085632325, 'f1_dc_time': 0.10285983085632325, 'mmd_rbf': 0.005727541695038477, 'mmd_rbf_time': 0.1497391

In [ ]:
0.73925 0.8126 0.7620000000000001 0.8126


In [ ]:
0.7620000000000001 0.8126 0.71085 0.7885000000000001


In [7]:
train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data('synthetic_1_50', path=r'data/graphs/datasets')
generated_graphs, generated_targets= get_corrupted_graph_data('synthetic_1_50',int(0*100))

In [9]:
train1_graphs[0].nodes(data=True)

NodeDataView({0: {'true_label': 1, 'label': 0}, 1: {'true_label': 0, 'label': 0}, 2: {'true_label': 2, 'label': 0}, 3: {'true_label': 0, 'label': 0}, 4: {'true_label': 1, 'label': 0}, 5: {'true_label': 1, 'label': 0}, 6: {'true_label': 0, 'label': 0}, 7: {'true_label': 2, 'label': 0}, 8: {'true_label': 2, 'label': 0}, 9: {'true_label': 1, 'label': 0}, 10: {'true_label': 2, 'label': 0}, 11: {'true_label': 0, 'label': 0}, 12: {'true_label': 2, 'label': 0}, 13: {'true_label': 2, 'label': 0}, 14: {'true_label': 1, 'label': 0}, 15: {'true_label': 2, 'label': 0}, 16: {'true_label': 0, 'label': 0}, 17: {'true_label': 0, 'label': 0}, 18: {'true_label': 0, 'label': 0}, 19: {'true_label': 0, 'label': 0}, 20: {'true_label': 1, 'label': 0}, 21: {'true_label': 0, 'label': 0}, 22: {'true_label': 1, 'label': 0}, 23: {'true_label': 0, 'label': 0}, 24: {'true_label': 2, 'label': 0}, 25: {'true_label': 2, 'label': 0}, 26: {'true_label': 0, 'label': 0}, 27: {'true_label': 2, 'label': 0}, 28: {'true_label

In [8]:
generated_graphs[0].nodes(data=True)

NodeDataView({0: {'label': 0}, 1: {'label': 0}, 2: {'label': 0}, 3: {'label': 0}, 4: {'label': 0}, 5: {'label': 0}, 6: {'label': 0}, 7: {'label': 0}, 8: {'label': 0}, 9: {'label': 0}, 10: {'label': 0}, 11: {'label': 0}, 12: {'label': 0}, 13: {'label': 0}, 14: {'label': 0}, 15: {'label': 0}, 16: {'label': 0}, 17: {'label': 0}, 18: {'label': 0}, 19: {'label': 0}, 20: {'label': 0}, 21: {'label': 0}, 22: {'label': 0}, 23: {'label': 0}, 24: {'label': 0}, 25: {'label': 0}, 26: {'label': 0}, 27: {'label': 0}, 28: {'label': 0}, 29: {'label': 0}})

# Mode Dropping


In [2]:
import pandas as pd
def get_mode_dropping_graph_data(name,percent):
   try:
     generated_graphs=pd.read_pickle(f'data/mode_dropping/train1/{name}/{name}_no_of_clusters_dropped_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/mode_dropping/train1/{name}/{name}_fraction_of_clusters_dropped_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
      try:
        target= int(g.nodes[0]['target'])
        targets.append(target)
      except:None
      
      for n,i in enumerate(g.nodes(data=True)):
        try:
          g.nodes[n]['label']=int(g.nodes[n]['label'])
        except: None
        try:
          g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
        except: None
        try:
            g.nodes[n]['attr']=[int(l) for l in list(g.nodes[n]['attr'])]
        except: None
        try:  del g.nodes[n]['target']
        except:None
       
   return  generated_graphs, np.array(targets)

def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict

In [10]:
generated=get_mode_dropping_graph_data('synthetic_1_50',0)

## Synthetic_1

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
r=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
rewiring_ratio=r
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in ['synthetic_1']:
        for split in splits:
            for percent in r:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,int(percent*100))
                if len(generated_targets)==0:
                    generated_targets=train1_targets
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)



## Synthetic_2

In [3]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
r=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
rewiring_ratio=r
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in r:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,int(percent*100))
                if len(generated_targets)==0:
                    generated_targets=train1_targets
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)



synthetic_2_50 0
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.82495 0.82805 0.8065000000000001 0.82805
{'activations_time': 0.040120665232340494, 'fid': 0.4230917669382945, 'fid_time': 0.043513154983520506, 'kid': 0.006520223617553711, 'kid_time': 0.3693932851155599, 'precision': 0.9776666666666666, 'recall': 0.9808333333333333, 'f1_pr': 0.979224913839528, 'precision_time': 0.10490263303120931, 'recall_time': 0.10490263303120931, 'f1_pr_time': 0.10490263303120931, 'density': 0.9774999999999998, 'coverage': 0.9488333333333332, 'f1_dc': 0.9617888190885137, 'density_time': 0.0737067699432373, 'coverage_time': 0.0737067699432373, 'f1_dc_time': 0.0737067699432373, 'mmd_rbf': 0.005727541695038477, 'mmd_rbf_time': 0.112223831

## Smiles Datasets


In [3]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['structural', 'auc_roc']
#leave it blank to compute all the structural structures 
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_dropping_graph_data(dataset_name+split,int(percent*100))
                    if len(generated_targets)==0:
                        generated_targets=train1_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)  
                    metrics=dict_mean(a)        
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

ames_50 0
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.8829444431748061 0.9113656100738168 0.7792162777765081 0.886062039597474
{'activations_time': 0.19936203956604004, 'fid': 2.7618283933656755, 'fid_time': 0.21286916732788086, 'kid': -0.08109832, 'kid_time': 0.27448296546936035, 'precision': 0.9729093050647821, 'recall': 0.9752650176678446, 'f1_pr': 0.9740957371290585, 'precision_time': 0.6114389896392822, 'recall_time': 0.6114389896392822, 'f1_pr_time': 0.6114389896392822, 'density': 1.0051040439733019, 'coverage': 0.9744797801334903, 'f1_dc': 0.9895650349761977, 'density_time': 0.5141198635101318, 'coverage_time': 0.5141198635101318, 'f1_dc_time': 0.5141198635101318, 'mmd_rbf

# Mode Collapse

In [8]:




import pandas as pd
def get_mode_collapse_graph_data(name,percent):
   try:
     generated_graphs=pd.read_pickle(f'data/mode_collapse/train1/{name}/{name}_no_of_clusters_collapsed_{percent}.pkl')
   except: generated_graphs=pd.read_pickle(f'data/mode_collapse/train1/{name}/{name}_fraction_of_clusters_collapsed_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
      try:
        target= int(g.nodes[0]['target'])
        targets.append(target)
      except:None
      
      for n,i in enumerate(g.nodes(data=True)):
        try:
          g.nodes[n]['label']=int(g.nodes[n]['label'])
        except: None
        try:
          g.nodes[n]['true_label']=int(g.nodes[n]['true_label'])
        except: None
        try:
            g.nodes[n]['attr']=[int(l) for l in list(g.nodes[n]['attr'])]
        except: None
        try:  del g.nodes[n]['target']
        except:None
       
   return  generated_graphs, np.array(targets)

## Sythetic_1

In [9]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
r=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
rewiring_ratio=r
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in ['synthetic_1']:
        for split in splits:
            for percent in r:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                if len(generated_targets)==0:
                    generated_targets=train1_targets
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)


synthetic_1_50 0
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.73925 0.8126 0.7620000000000001 0.8126
{'activations_time': 0.04231179555257161, 'fid': 10.09712711137854, 'fid_time': 0.04388157526652018, 'kid': 91.85098775227864, 'kid_time': 0.48769524892171223, 'precision': 0.9643333333333334, 'recall': 0.9778333333333332, 'f1_pr': 0.9709917770005424, 'precision_time': 0.13645876248677571, 'recall_time': 0.13645876248677571, 'f1_pr_time': 0.13645876248677571, 'density': 1.0061333333333333, 'coverage': 0.9761666666666666, 'f1_dc': 0.9899602898144073, 'density_time': 0.10188647905985514, 'coverage_time': 0.10188647905985514, 'f1_dc_time': 0.10188647905985514, 'mmd_rbf': 0.005290028142432372, 'mmd_rbf_time': 0.125386015574

## Synthetic_2

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural','auc_roc']
generators=['swingnn','gdss']
splits=['_50']
r=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
rewiring_ratio=r
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in r:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                if len(generated_targets)==0:
                    generated_targets=train1_targets
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)


## Smiles Datasets

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn', 'structural', 'auc_roc']
#leave it blank to compute all the structural structures 
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=get_mode_collapse_graph_data(dataset_name+split,int(percent*100))
                    if len(generated_targets)==0:
                        generated_targets=train1_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)  
                    metrics=dict_mean(a)        
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

ames_50 hiervae
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Cannot compute the classifier based metrics
Now computing structural based metrics


## Additionally train2 versus train2

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn', 'structural', 'auc_roc']
#leave it blank to compute all the structural structures 
rewiring_ratio=[0,0.05, 0.10, 0.15, 0.25, 0.33, 0.45, 0.50, 0.65, 0.70,0.75,0.85,0.90,0.95, 1]
metrics={}
for generator in ['train2']:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            for percent in rewiring_ratio:
                try:
                    a=[]
                    print(dataset_name+split, percent)
                    train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                    generated_graphs, generated_targets=train2_graphs , train2_targets
                    splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                    print('Now computing classifier based metrics')
                    for run in range(15):
                        metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                        a.append(metrics)  
                    metrics=dict_mean(a)        
                    metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                    metrics['dataset']=dataset_name+split
                    metrics['generator_name']=generator
                    metrics['degree_of_perturbation']=percent
                    print(metrics)
                except:continue

# SWINGNN as we increase the number of training epochs

In [6]:

import pandas as pd
from glob import glob
from sklearn.utils import shuffle                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.utils import remove_empty_graphs_and_targets
import numpy as np

def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
      try:
        mean_dict[key] = sum(float(d[key]) for d in dict_list) / len(dict_list)
      except: mean_dict[key]=dict_list[0][key]
    return mean_dict 
def get_swingnn_graph_data(name,percent,split,path='data/SWINGNN_experiments/'):
     path+=f'{percent}-percent/'
     path_postives=path+r'{}/{}/pos/*.pkl'.format(name,name+split)
     files = glob(path_postives)
     generated_pos_graphs=pd.read_pickle(files[0])
     path_negatives=path+r'{}/{}/neg/*.pkl'.format(name,name+split)
     files = glob(path_negatives)
     generated_neg_graphs=pd.read_pickle(files[0])
     generated_graphs,generated_targets = generated_pos_graphs + generated_neg_graphs, [1]*len(generated_pos_graphs)+[0]*len(generated_neg_graphs)

     return  generated_graphs, np.array(generated_targets)

In [7]:
from glob import glob
from sklearn.utils import shuffle                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.utils import remove_empty_graphs_and_targets
import numpy as np

def get_swingnn_molecular_data(name,percent, path='data/SWINGNN_experiments/', generator_name='stgg',return_smiles=False):
     path+=f'{percent}-percent/'
     pos_list, neg_list=[],[]
     path_postives=path+r'{}/{}/pos/*.txt'.format(name,name+'_50')
     files = glob(path_postives)
     
     with open(files[0]) as my_file:
          for line in my_file:
               pos_list.append(line.strip())

     path_negatives=path+r'{}/{}/neg/*.txt'.format(name,name+'_50')
     files = glob(path_negatives)
     
     with open(files[0]) as my_file:
          for line in my_file:
               neg_list.append(line.strip())

     pos_graphs=list_of_smiles_to_nx_graphs(pos_list)
     neg_graphs=list_of_smiles_to_nx_graphs(neg_list)
     #print(len(neg_graphs))
     generated_graphs,generated_targets = pos_graphs + neg_graphs, [1]*len(pos_graphs)+[0]*len(neg_graphs)
     #generated_graphs, generated_targets = shuffle(generated_graphs, generated_targets)
     generated_graphs, generated_targets = remove_empty_graphs_and_targets(generated_graphs, generated_targets)
     if return_smiles:
          return  generated_graphs, generated_targets,pos_list+neg_list
     return  generated_graphs, np.array(generated_targets)

## Molecular graphs

### nn and structural 

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_25','_33','_40','_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['swingnn']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
split='_50'
for generator in generators:
    for dataset_name in current_smiles_datasets:
             for percent in experiments:
                a=[]
                print(dataset_name+split)
                train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                generated_graphs, generated_targets=get_swingnn_molecular_data(dataset_name,percent)
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
                print('Now computing classifier based metrics')
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)  
                metrics=dict_mean(a)        
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

### train2 vs train1

In [ ]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train1']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
split='_50'
for generator in generators:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            a=[]
            print(dataset_name+split)
            train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
            generated_graphs, generated_targets=train1_graphs , train1_targets
            splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets , test_graphs, test_targets, generated_graphs, generated_targets]
            print('Now computing classifier based metrics')
            for run in range(15):
                metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                a.append(metrics)  
            metrics=dict_mean(a)        
            metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

### molecular

In [8]:
from evaluation.moses.metrics.metrics import get_all_metrics
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split1='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['swingnn']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            for percent in experiments:
                print(dataset_name+split1)
                graphs, splits =get_data(dataset_name+split1, return_smiles=True)
                train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
                _,generated_targets, generated_smiles=get_swingnn_molecular_data(dataset_name,percent,return_smiles=True) 
                metrics=get_all_metrics(gen=generated_smiles,train=train2_smiles,test=train2_smiles)
                metrics['dataset']=dataset_name+split1
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100 
                print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.992, 'unique@10000': 0.9752650176678446, 'FCD/Test': 17.196146057508543, 'SNN/Test': 0.27051011988074813, 'Frag/Test': 0.1531511401333041, 'Scaf/Test': 0.05274522872358611, 'FCD/TestSF': 37.4445580256326, 'SNN/TestSF': 0.19734603975408443, 'Frag/TestSF': 0.27915602040639564, 'Scaf/TestSF': 4.1041894040527716e-05, 'IntDiv': 0.9190337370237776, 'IntDiv2': 0.9022144090456522, 'Filters': 0.5127601099332548, 'logP': 0.19126302709069487, 'SA': 1.4890440133925626, 'QED': 0.08059893151943091, 'weight': 54.34200942285044, 'Novelty': 0.9967793880837359, 'dataset': 'ames_50', 'generator_name': 'swingnn', 'numer_of_epochs_trained': 250.0}
ames_50
{'valid': 1.0, 'unique@1000': 0.987, 'unique@10000': 0.9733019238319591, 'FCD/Test': 15.621304290928059, 'SNN/Test': 0.28040955265129036, 'Frag/Test': 0.19670744762450332, 'Scaf/Test': 0.07717364804524973, 'FCD/TestSF': 35.64221637716807, 'SNN/TestSF': 0.21063697174583457, 'Frag/TestSF': 0.4119607673591432, 'Scaf/Te

#### train1


In [9]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train1']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            metrics=get_all_metrics(gen=train1_smiles,train=train2_smiles,test=train2_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9996073812328229, 'FCD/Test': 0.8156919603444166, 'SNN/Test': 0.575357792673038, 'Frag/Test': 0.9878820451873492, 'Scaf/Test': 0.8593618010840082, 'FCD/TestSF': 22.270570217406373, 'SNN/TestSF': 0.33544746274439363, 'Frag/TestSF': -0.06553137100764106, 'Scaf/TestSF': 0.13905682249091533, 'IntDiv': 0.9105206593714188, 'IntDiv2': 0.8919648544524341, 'Filters': 0.5551629367883785, 'logP': 0.08376508833922258, 'SA': 0.04752492131394497, 'QED': 0.0039007607322941876, 'weight': 4.032424028268554, 'Novelty': 0.997643362136685, 'dataset': 'ames_50', 'generator_name': 'train1'}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9943661971830986, 'unique@10000': 0.9943661971830986, 'FCD/Test': 3.6841985920171965, 'SNN/Test': 0.5081955164146256, 'Frag/Test': 0.966294915338154, 'Scaf/Test': 0.5502519513284054, 'FCD/TestSF': 19.44386748202205, 'SNN/TestSF': 0.3730856233761764, 'Frag/TestSF': -0.10547896046759009, 'Scaf/TestSF': 0.0249430999

In [10]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            metrics=get_all_metrics(gen=train2_smiles,train=train2_smiles,test=train2_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9988221436984688, 'FCD/Test': 2.7711166694643907e-13, 'SNN/Test': 1.0, 'Frag/Test': 1, 'Scaf/Test': 1, 'FCD/TestSF': 22.184285088577205, 'SNN/TestSF': 0.33677946461981123, 'Frag/TestSF': -0.02581333437055111, 'Scaf/TestSF': 0.20333785426573803, 'IntDiv': 0.9102053951183938, 'IntDiv2': 0.8921645781772736, 'Filters': 0.5618374558303887, 'logP': 0.0, 'SA': 0.0, 'QED': 0.0, 'weight': 0.0, 'Novelty': 0.00039308176100628933, 'dataset': 'ames_50', 'generator_name': 'train2'}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9957805907172996, 'unique@10000': 0.9957805907172996, 'FCD/Test': 9.947598300641403e-14, 'SNN/Test': 1.0, 'Frag/Test': 1, 'Scaf/Test': 1, 'FCD/TestSF': 20.86298113850406, 'SNN/TestSF': 0.3738190557404242, 'Frag/TestSF': 0.06172323226684595, 'Scaf/TestSF': 0.02450489523326016, 'IntDiv': 0.8879810418473504, 'IntDiv2': 0.8712773076208551, 'Filters': 0.7848101265822784, 'logP': 0.0, 'SA': 0.0, 'QED': 0.0, 'weight': 0.

In [11]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train1/2+train2/2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            half=len(train1_smiles)//2
            train3_smiles=train1_smiles[:half] + train2_smiles[:half]
            metrics=get_all_metrics(gen=train3_smiles,train=train2_smiles,test=train2_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9972505891594659, 'FCD/Test': 1.4296307269705935, 'SNN/Test': 0.80414747127214, 'Frag/Test': 0.9769171525419632, 'Scaf/Test': 0.9058604214675904, 'FCD/TestSF': 24.652601465116575, 'SNN/TestSF': 0.3295319009622016, 'Frag/TestSF': -0.014604840934317576, 'Scaf/TestSF': 0.13555299909832097, 'IntDiv': 0.9005626242852879, 'IntDiv2': 0.8800491646635825, 'Filters': 0.4615082482325216, 'logP': 0.13850204932346513, 'SA': 0.06508225753225652, 'QED': 0.02354588307221548, 'weight': 11.541142909838637, 'Novelty': 0.4990153603781016, 'dataset': 'ames_50', 'generator_name': 'train1/2+train2/2'}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9929577464788732, 'unique@10000': 0.9929577464788732, 'FCD/Test': 3.722576671925424, 'SNN/Test': 0.74866054233321, 'Frag/Test': 0.9741186915938969, 'Scaf/Test': 0.7575674931332002, 'FCD/TestSF': 19.48522451637382, 'SNN/TestSF': 0.378976951418838, 'Frag/TestSF': 0.21368685111121732, 'Scaf/TestSF': 0.0076

## Synthetic graphs 

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['structural', 'auc_roc']
generators=['swingnn']
splits=['_25','_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['swingnn']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in experiments:
                a=[]
                print(dataset_name+split, percent)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_swingnn_graph_data(dataset_name,percent,split)
                splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                for run in range(15):
                    metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                    a.append(metrics)
                metrics=dict_mean(a)
                metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

 ### Synthetic train2 vs train2

In [ ]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn','structural', 'auc_roc']
generators=['train2']
splits=['_25','_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['train2']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            a=[]
            print(dataset_name+split)
            train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
            generated_graphs, generated_targets=train2_graphs,train2_targets
            splits_for_auc_roc=[train2_graphs , train2_targets,train1_graphs , train1_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            for run in range(15):
                metrics=evaluate(train2_graphs, generated_graphs, device,  ['nn'], structural_statistic,*splits_for_auc_roc)
                a.append(metrics)
            metrics=dict_mean(a)
            metrics.update(evaluate(train2_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc))
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)


synthetic_1_25
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7019 0.7620000000000001 0.7061000000000001 0.7620000000000001
{'activations_time': 0.09806990623474121, 'fid': 0.22037899701926023, 'fid_time': 0.32657504081726074, 'kid': -0.002235651, 'kid_time': 0.1197969913482666, 'precision': 0.97, 'recall': 0.99, 'f1_pr': 0.9799079602248955, 'precision_time': 1.095655918121338, 'recall_time': 1.095655918121338, 'f1_pr_time': 1.095655918121338, 'density': 1.026, 'coverage': 0.98, 'f1_dc': 1.0024825875115901, 'density_time': 0.22144103050231934, 'coverage_time': 0.22144103050231934, 'f1_dc_time': 0.22144103050231934, 'mmd_rbf': 0.00850243540480733, 'mmd_rbf_time': 0.36985182762145996, 'mmd_linear': 0.03589476, 'mmd_linear_time': 0.098114013671875, 'wl_mmd': 0.01, 'wl_mmd_time': 0.24774694442749023, 'nspdk_mmd': 0.008445549078421655, 'nspdk_mmd_time': 4.805341005325317, 'degree_mmd': 0.00026336114711855707, 'degree_m